In [1]:
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Embedding, Flatten
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# import keras_tuner as kt
import sklearn as sk

In [2]:
def read_data(filename):
    df = pd.read_csv(filename, dtype='float64')
    fieldnames = list(df.columns)
    data = np.array(df.to_dict(orient='records'))
    return data, fieldnames
data, fieldnames = read_data('dataset/rl_data_final_cont.csv')

In [3]:
print('All Fieldnames:', [i for i in enumerate(fieldnames)])

All Fieldnames: [(0, 'bloc'), (1, 'icustayid'), (2, 'charttime'), (3, 'gender'), (4, 'age'), (5, 'elixhauser'), (6, 're_admission'), (7, 'died_in_hosp'), (8, 'died_within_48h_of_out_time'), (9, 'mortality_90d'), (10, 'delay_end_of_record_and_discharge_or_death'), (11, 'Weight_kg'), (12, 'GCS'), (13, 'HR'), (14, 'SysBP'), (15, 'MeanBP'), (16, 'DiaBP'), (17, 'RR'), (18, 'SpO2'), (19, 'Temp_C'), (20, 'FiO2_1'), (21, 'Potassium'), (22, 'Sodium'), (23, 'Chloride'), (24, 'Glucose'), (25, 'BUN'), (26, 'Creatinine'), (27, 'Magnesium'), (28, 'Calcium'), (29, 'Ionised_Ca'), (30, 'CO2_mEqL'), (31, 'SGOT'), (32, 'SGPT'), (33, 'Total_bili'), (34, 'Albumin'), (35, 'Hb'), (36, 'WBC_count'), (37, 'Platelets_count'), (38, 'PTT'), (39, 'PT'), (40, 'INR'), (41, 'Arterial_pH'), (42, 'paO2'), (43, 'paCO2'), (44, 'Arterial_BE'), (45, 'Arterial_lactate'), (46, 'HCO3'), (47, 'mechvent'), (48, 'Shock_Index'), (49, 'PaO2_FiO2'), (50, 'median_dose_vaso'), (51, 'max_dose_vaso'), (52, 'input_total'), (53, 'input_4

In [4]:
all_features = fieldnames[3:7] + fieldnames[11:50] + [fieldnames[56]] + fieldnames[57:59] + fieldnames[50:52] + fieldnames[59:61]
#features = ['Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate', 'HCO3', 'mechvent', 'Shock_Index', 'PaO2_FiO2', 'cumulated_balance', 'SOFA', 'SIRS', 'median_dose_vaso', 'max_dose_vaso', 'vaso_input', 'iv_input']
print(f'All Features({len(all_features)}):', [i for i in enumerate(all_features)])

All Features(50): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission'), (4, 'Weight_kg'), (5, 'GCS'), (6, 'HR'), (7, 'SysBP'), (8, 'MeanBP'), (9, 'DiaBP'), (10, 'RR'), (11, 'SpO2'), (12, 'Temp_C'), (13, 'FiO2_1'), (14, 'Potassium'), (15, 'Sodium'), (16, 'Chloride'), (17, 'Glucose'), (18, 'BUN'), (19, 'Creatinine'), (20, 'Magnesium'), (21, 'Calcium'), (22, 'Ionised_Ca'), (23, 'CO2_mEqL'), (24, 'SGOT'), (25, 'SGPT'), (26, 'Total_bili'), (27, 'Albumin'), (28, 'Hb'), (29, 'WBC_count'), (30, 'Platelets_count'), (31, 'PTT'), (32, 'PT'), (33, 'INR'), (34, 'Arterial_pH'), (35, 'paO2'), (36, 'paCO2'), (37, 'Arterial_BE'), (38, 'Arterial_lactate'), (39, 'HCO3'), (40, 'mechvent'), (41, 'Shock_Index'), (42, 'PaO2_FiO2'), (43, 'cumulated_balance'), (44, 'SOFA'), (45, 'SIRS'), (46, 'median_dose_vaso'), (47, 'max_dose_vaso'), (48, 'vaso_input'), (49, 'iv_input')]


In [6]:
static_features = fieldnames[3:7]
print(f'Static Features({len(static_features)}):', [i for i in enumerate(static_features)])

Static Features(4): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission')]


In [7]:
labels = fieldnames[3:7] + fieldnames[11:50] + [fieldnames[56]] + fieldnames[57:59]
#labels = ['Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate', 'HCO3', 'mechvent', 'Shock_Index', 'PaO2_FiO2', 'cumulated_balance', 'SOFA', 'SIRS']
print(f'labels({len(labels)}):', [i for i in enumerate(labels)])

labels(46): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission'), (4, 'Weight_kg'), (5, 'GCS'), (6, 'HR'), (7, 'SysBP'), (8, 'MeanBP'), (9, 'DiaBP'), (10, 'RR'), (11, 'SpO2'), (12, 'Temp_C'), (13, 'FiO2_1'), (14, 'Potassium'), (15, 'Sodium'), (16, 'Chloride'), (17, 'Glucose'), (18, 'BUN'), (19, 'Creatinine'), (20, 'Magnesium'), (21, 'Calcium'), (22, 'Ionised_Ca'), (23, 'CO2_mEqL'), (24, 'SGOT'), (25, 'SGPT'), (26, 'Total_bili'), (27, 'Albumin'), (28, 'Hb'), (29, 'WBC_count'), (30, 'Platelets_count'), (31, 'PTT'), (32, 'PT'), (33, 'INR'), (34, 'Arterial_pH'), (35, 'paO2'), (36, 'paCO2'), (37, 'Arterial_BE'), (38, 'Arterial_lactate'), (39, 'HCO3'), (40, 'mechvent'), (41, 'Shock_Index'), (42, 'PaO2_FiO2'), (43, 'cumulated_balance'), (44, 'SOFA'), (45, 'SIRS')]


In [8]:
# Just to indicate, not used.
action_names = fieldnames[50:52] + fieldnames[59:61]
print(f'Action Names({len(action_names)}):', [i for i in enumerate(action_names)])

Action Names(4): [(0, 'median_dose_vaso'), (1, 'max_dose_vaso'), (2, 'vaso_input'), (3, 'iv_input')]


In [9]:
def get_padded_features_and_labels(data):
    _x = []
    _y = []
    current_id = data[0]['icustayid']
    temp_x = []
    temp_y = []
    for i in range(len(data)-1):
        if data[i]['icustayid'] != data[i+1]['icustayid']:
            continue
        if data[i]['icustayid'] != current_id:
            current_id = data[i]['icustayid']
            _x.append(temp_x)
            _y.append(temp_y)
            temp_x = []
            temp_y = []
        temp_x.append([data[i][k] for k in all_features])
        temp_y.append([data[i+1][k] for k in labels])
    if temp_x != []:
        _x.append(temp_x)
        _y.append(temp_y)
    _x = tf.keras.preprocessing.sequence.pad_sequences(_x, padding='pre', dtype='float64', value=0)
    _y = tf.keras.preprocessing.sequence.pad_sequences(_y, padding='pre', dtype='float64', value=0)
    _x = np.array(_x)
    _y = np.array(_y)
    _x = _x.reshape(_x.shape[0]*_x.shape[1], _x.shape[2])
    _y = _y.reshape(_y.shape[0]*_y.shape[1], _y.shape[2])
    return _x, _y

def get_features_and_labels(data):
    _x = []
    _y = []
    for i in range(len(data)-1):
        if data[i]['icustayid'] != data[i+1]['icustayid']:
            continue
 
        _x.append([data[i][k] for k in all_features])
        _y.append([data[i+1][k] for k in labels])
    return np.array(_x), np.array(_y)

def get_comparison_testing_data(data):
    _X, _y = get_padded_features_and_labels(data)
    _, _X_te, _, _y_te = train_test_split(_X, _y, shuffle=False, test_size=0.2)
    X_te_comp = np.array([i for i in _X_te if not np.array_equal(i, np.zeros(50))])
    y_te_comp = np.array([i for i in _y_te if not np.array_equal(i, np.zeros(46))])
    return X_te_comp, y_te_comp


In [10]:
X, y = get_padded_features_and_labels(data)

print(X.shape, y.shape)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, shuffle=False, test_size=0.2)

X_te_comp, y_te_comp = get_comparison_testing_data(data)

# Normalize the features
scaler_X = sk.preprocessing.StandardScaler().fit(X)
X_tr_scaled = scaler_X.transform(X_tr)
X_te_scaled = scaler_X.transform(X_te)
# X_scaled = scaler_X.transform(X)



# Normalize the labels
scaler_y = sk.preprocessing.StandardScaler().fit(y)
y_tr_scaled = scaler_y.transform(y_tr)
y_te_scaled = scaler_y.transform(y_te)

#print(X_tr_scaled.shape, X_te_scaled.shape, X_scaled.shape)#, y_tr_scaled.shape, y_te_scaled.shape)

(393072, 50) (393072, 46)


In [11]:
# Extract Static features

X_tr_st_scaled = X_tr_scaled[:, :4]

X_te_st_scaled = X_te_scaled[:, :4]

# X_te_st_unpadded_scaled = X_te_unpadded_scaled[:, :4]

print("X_tr_st_scaled shape:: ", X_tr_st_scaled.shape)
print("X_te_st_scaled shape:: ", X_te_st_scaled.shape)
# print("X_te_st_unpadded_scaled shape:: ", X_te_st_unpadded_scaled.shape)


X_tr_st_scaled shape::  (314457, 4)
X_te_st_scaled shape::  (78615, 4)


# Hyperparameter Tuning

In [11]:
def model_builder(hp):
    def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
        # Attention and Normalization
        x = LayerNormalization(epsilon=1e-6)(inputs)
        x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
        x = Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = LayerNormalization(epsilon=1e-6)(res)
        x = Dense(ff_dim, activation="relu")(x)
        x = Dropout(dropout)(x)
        x = Dense(inputs.shape[-1])(x)
        return x + res

    def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
        static_inputs = tf.keras.Input(shape=(len(static_features)))
        mixed_inputs = Input(shape=input_shape)
        x = mixed_inputs

        # Build Encoder Layers
        for _ in range(num_layers):
            x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        # Final Layer
        x = Flatten()(x)
        static_outputs = (static_inputs)
        mixed_outputs = Dense(42, activation="linear")(x)
        main_outputs = tf.keras.layers.concatenate([static_outputs, mixed_outputs])

        model = Model([static_inputs, mixed_inputs], main_outputs)
        return model

    # Model Parameters
    input_shape = (1, 50) 
    hp_units1 = hp.Int('units1', min_value=64, max_value=1280, step=64)
    head_size = hp_units1
    hp_units2 = hp.Int('units2', min_value=2, max_value=16, step=2)
    num_heads = hp_units2
    hp_units3 = hp.Int('units3', min_value=64, max_value=512, step=32)
    ff_dim = hp_units3
    num_layers = 2
    dropout = 0.0

    # Build and Compile Model
    model = build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout)
    hp_lr = hp.Choice('learning_rate', values=[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001])
    opt = keras.optimizers.Adam(learning_rate=hp_lr) #set=0.005, #default=0.001
    model.compile(optimizer=opt, loss="mean_squared_error")

#     model.summary()
    return model

In [14]:
X_tr_reshaped = X_tr_scaled.reshape((X_tr_scaled.shape[0], 
                                    1, X_tr_scaled.shape[1]))
X_te_reshaped = X_te_scaled.reshape((X_te_scaled.shape[0], 
                                    1, X_te_scaled.shape[1]))

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=50,
                     factor=3)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             patience=5)
tuner.search([X_tr_st_scaled, X_tr_reshaped], y_tr_scaled, epochs=50, batch_size=512,
                    shuffle=False,
                    validation_data=([X_te_st_scaled, X_te_reshaped], y_te_scaled), 
                    callbacks=[earlystop])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 90 Complete [00h 01m 59s]
val_loss: 1.9870116710662842

Best val_loss So Far: 0.13094863295555115
Total elapsed time: 02h 15m 47s


In [15]:
print(best_hps.get('units1'))
print(best_hps.get('units2'))
print(best_hps.get('units3'))
print(best_hps.get('learning_rate'))

64
16
288
0.0005


Transformer:: 64(head size), 16(num heads), 288(ff dim), 0.0005(lr)

# End Tuning

In [12]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs
    
    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Build Encoder Layers
    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    # Final Layer
    x = Flatten()(x)
    outputs = Dense(46, activation="linear")(x)
    
    model = Model(inputs, outputs)
    return model

# Model Parameters
input_shape = (1, 50)
head_size = 64
num_heads = 16
ff_dim = 288
num_layers = 2
dropout = 0.0

# Build and Compile Model
model = build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout)
opt = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer="adam", loss="mean_squared_error")

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 50)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 1, 50)        100         input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 1, 50)        207922      layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1, 50)        0           multi_head_attention[0][0]   

In [13]:
X_tr_reshaped = X_tr_scaled.reshape((X_tr_scaled.shape[0], 
                                    1, X_tr_scaled.shape[1]))
X_te_reshaped = X_te_scaled.reshape((X_te_scaled.shape[0], 
                                    1, X_te_scaled.shape[1]))

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             mode='min',
                                             min_delta=0,
                                             patience=7,
                                             verbose=1)

model.fit(X_tr_reshaped, y_tr_scaled, 
          validation_data=(X_te_reshaped, y_te_scaled), 
          epochs=100, batch_size=1024, verbose=1, callbacks=[earlystop])

Epoch 1/100
308/308 [==============================] - 9s 16ms/step - loss: 0.2512 - val_loss: 0.1564
Epoch 2/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1663 - val_loss: 0.1448
Epoch 3/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1545 - val_loss: 0.1403
Epoch 4/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1494 - val_loss: 0.1390
Epoch 5/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1458 - val_loss: 0.1367
Epoch 6/100
308/308 [==============================] - 4s 14ms/step - loss: 0.1435 - val_loss: 0.1361
Epoch 7/100
308/308 [==============================] - 4s 14ms/step - loss: 0.1419 - val_loss: 0.1362
Epoch 8/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1409 - val_loss: 0.1345
Epoch 9/100
308/308 [==============================] - 5s 15ms/step - loss: 0.1395 - val_loss: 0.1357
Epoch 10/100
308/308 [==============================] - 4s 14ms/step - loss: 0.138

In [14]:
def get_prediction_mse(model, X, y):
    _X = scaler_X.transform(X)
    _y = scaler_y.transform(y)
    _X_st = _X[:, :4]
    _X = _X.reshape((_X.shape[0], 1, _X.shape[1]))
    prediction = model.predict(_X)
#     prediction = scaler_y.inverse_transform(prediction)
    prediction[:, :4] = _X_st#[:, :4]
#     prediction = prediction[:, :46]
    error = sk.metrics.mean_squared_error(prediction, _y)
    return error

In [15]:
print(get_prediction_mse(model, X_tr, y_tr))
print(get_prediction_mse(model, X_te, y_te))
print(get_prediction_mse(model, X_te_comp, y_te_comp))

0.13034162485934567
0.13087883175968082
0.20063597817065362


In [16]:
model.save('saved_models/transformer')

INFO:tensorflow:Assets written to: saved_models/transformer\assets


INFO:tensorflow:Assets written to: saved_models/transformer\assets


In [17]:
temp_model = tf.keras.models.load_model('saved_models/transformer')

In [18]:
print(get_prediction_mse(temp_model, X_tr, y_tr))
print(get_prediction_mse(temp_model, X_te, y_te))
print(get_prediction_mse(temp_model, X_te_comp, y_te_comp))

0.13034162485934567
0.13087883175968082
0.20063597817065362
